In [1]:
import random, requests, hashlib, base64, time
import json
import re
#from HTMLParser import HTMLParser
from bs4 import BeautifulSoup, NavigableString, Tag
from bs4 import BeautifulSoup as BS
from bs4 import Comment
import csv
import sys
import unicodedata
import datetime
#import arrow
import urllib
import pandas as pd
import numpy as np
import urllib
import urllib.request
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
reader = pd.read_csv("free-7-million-company-dataset/companies_sorted.csv",low_memory=True, chunksize=10**4)
df = pd.concat([x for x in reader], ignore_index=True)
print('complete')

complete


In [3]:
df.sort_values(by='current employee estimate', ascending=False).head()

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
0,5872184,ibm,ibm.com,1911.0,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906
1,4425416,tata consultancy services,tcs.com,1968.0,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369
2,21074,accenture,accenture.com,1989.0,information technology and services,10001+,"dublin, dublin, ireland",ireland,linkedin.com/company/accenture,190689,455768
3,2309813,us army,goarmy.com,1800.0,military,10001+,"alexandria, virginia, united states",united states,linkedin.com/company/us-army,162163,445958
4,1558607,ey,ey.com,1989.0,accounting,10001+,"london, greater london, united kingdom",united kingdom,linkedin.com/company/ernstandyoung,158363,428960


In [ ]:
#value counts
print(df['size range'].value_counts())
print()
print(df['country'].value_counts())
print()
print(df['industry'].value_counts())
print()
print(df['year founded'].value_counts(bins=10))
print()
print(df['current employee estimate'].value_counts(bins=10))

In [12]:
df[df['name'].str.contains('fotio', na=False, regex=False)]

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
1386515,4131309,fotiou frames limited,fotiou.com,NaN,consumer goods,1 - 10,"vaughan, ontario, canada",canada,linkedin.com/company/fotiou-frames-limited,5,7
4831970,689172,fotio,fotio.co,2012.0,photography,1 - 10,NaN,NaN,linkedin.com/company/fotio,1,1
4899621,39906,f.g.fotiou developments & constructions ltd,fgfotiou.com,1980.0,construction,1 - 10,"mesa geitonia, limassol, cyprus",cyprus,linkedin.com/company/f-g-fotiou-developments-&...,1,3


In [5]:
#print(df[df.index < 10])
dfa = df[df['name'] == 'fotio']
for index, row in dfa.iterrows():
    try:
        url2 ='https://' + str(row['domain'])
        url = urllib.request.urlopen(url2)
        s = url.read()
        soup = BeautifulSoup(s, 'html.parser')
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))
        print('start')
        for c in comments:
            if "Squarespace" in c:
                print("YESS")
                break
        print('finish')    
        print(url2)
    
    except:
        try:
            url2 ='http://' + str(row['domain'])
            print(url2)
            url = urllib.request.urlopen(url2)
            s = url.read()
            soup = BeautifulSoup(s, 'html.parser')
            comments = soup.find_all(string=lambda text: isinstance(text, Comment))
            for c in comments:
                if "Squarespace" in c:
                    print("YESS")
                    break
        except:
            pass
        
     

start
YESS
finish
https://fotio.co


In [ ]:
   
        print(url2)
        url2='https://' + str(row['domain'])
        with urllib.request.urlopen(url2) as url:
            s = url.read()
            #data = json.loads(url.read())
            #pr = requests.get(url2)
            #data = json.loads(s)
            #print(s)
            soup = BeautifulSoup(s, 'html.parser')
            comments = soup.find_all(string=lambda text: isinstance(text, Comment))
            for c in comments:
                print(c)
                print("===========")
                c.extract()
                if "Squarespace" in c:
                    #element = soup.find("script", data-name="static-context")
                    #print(element.attrs["Static.SQUARESPACE_CONTEXT"])
                    break
    except:
        print(url2)
        url2='http://' + str(row['domain'])
        with urllib.request.urlopen(url2) as url:
            s = url.read()
            #data = json.loads(url.read())
            #pr = requests.get(url2)
            #data = json.loads(s)
            #print(s)
            soup = BeautifulSoup(s, 'html.parser')
            comments = soup.find_all(string=lambda text: isinstance(text, Comment))
            for c in comments:
                print(c)
                print("===========")
                c.extract()
                if "Squarespace" in c:
                    #element = soup.find("script", data-name="static-context")
                    #print(element.attrs["Static.SQUARESPACE_CONTEXT"])
                    break

In [ ]:
url2= 'https://www.fotio.co/'
with urllib.request.urlopen(url2) as url:
    s = url.read()
    #data = json.loads(url.read())
    #pr = requests.get(url2)
    #data = json.loads(s)
    #print(s)
    soup = BeautifulSoup(s, 'html.parser')
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    for c in comments:
        print(c)
        print("===========")
        c.extract()
        if "Squarespace" in c:
            #element = soup.find("script", data-name="static-context")
            #print(element.attrs["Static.SQUARESPACE_CONTEXT"])
            break
        